# The heart in the Tenakh

## Table of content <a class="anchor" id="TOC"></a>

* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Load Text-Fabric app and data</a>
* <a href="#bullet3">3 - Performing the queries</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to TOC](#TOC)

The Hebrew word for heart can be either lb or lbb. This Jupyter NoteBook investigates occurances per book.

# 2 - Load Text-Fabric app and data <a class="anchor" id="bullet2"></a>
##### [Back to TOC](#TOC)

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# Loading the Text-Fabric code
# Note: it is assumed Text-Fabric is installed in your environment.
from tf.fabric import Fabric
from tf.app import use

In [6]:
# load the BHSL app and data
BHS = use ("etcbc/BHSA",hoist=globals())

**Locating corpus resources ...**

Name,# of nodes,# slots/node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


Note: Thefeature documentation can be found at [ETCBC GitHub](https://github.com/ETCBC/bhsa/blob/master/docs/features/0_home.md) 

In [16]:
# The following will push the Text-Fabric stylesheet to this notebook (to facilitate proper display with notebook viewer)
BHS.dh(BHS.getCss())

# 3 - Performing the queries <a class="anchor" id="bullet3"></a>
##### [Back to TOC](#TOC)

In [7]:
LevQuery = '''
book
  chapter
     verse
       word lex=LBB/|LB/ 
'''

LevResults = BHS.search(LevQuery)

  0.45s 861 results


In [15]:
# Library to format table
from tabulate import tabulate

ResultDict = {}
for Item in LevResults:
    Book=F.book.v(Item[0])
    Lex=F.lex.v(Item[3])
    if Book in ResultDict:
        # If it exists, add the count to the existing value
        ResultDict[Book][0]+=1
        if Lex=='LB/':
            ResultDict[Book][1]+=1
        else:
            ResultDict[Book][2]+=1
    else:
        # If it doesn't exist, initialize the count as the value
        if Lex=='LB/':
            ResultDict[Book]=[1,1,0]
        else:
            ResultDict[Book]=[1,0,1]
            

# Convert the dictionary into a list of key-value pairs and sort it according to frequency
UnsortedTableData = [[key, value[0],value[1],value[2]] for key, value in ResultDict.items()]
TableData= sorted(UnsortedTableData, key=lambda row: row[1], reverse=True)

# Produce the table
headers = ["Book","Total","LB","LBB"]
print(tabulate(TableData, headers=headers, tablefmt='fancy_grid'))


╒═══════════════╤═════════╤══════╤═══════╕
│ Book          │   Total │   LB │   LBB │
╞═══════════════╪═════════╪══════╪═══════╡
│ Psalmi        │     137 │  102 │    35 │
├───────────────┼─────────┼──────┼───────┤
│ Proverbia     │      99 │   97 │     2 │
├───────────────┼─────────┼──────┼───────┤
│ Jeremia       │      66 │   58 │     8 │
├───────────────┼─────────┼──────┼───────┤
│ Deuteronomium │      51 │    4 │    47 │
├───────────────┼─────────┼──────┼───────┤
│ Jesaia        │      49 │   31 │    18 │
├───────────────┼─────────┼──────┼───────┤
│ Exodus        │      47 │   46 │     1 │
├───────────────┼─────────┼──────┼───────┤
│ Ezechiel      │      47 │   41 │     6 │
├───────────────┼─────────┼──────┼───────┤
│ Chronica_II   │      44 │   16 │    28 │
├───────────────┼─────────┼──────┼───────┤
│ Ecclesiastes  │      42 │   41 │     1 │
├───────────────┼─────────┼──────┼───────┤
│ Reges_I       │      37 │   14 │    23 │
├───────────────┼─────────┼──────┼───────┤
│ Samuel_I 